In [1]:
# Day_02_01_mnist_cnn.py
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import  input_data

In [2]:
def mnist_cnn_1():
    mnist = input_data.read_data_sets('mnist')

    ph_x = tf.placeholder(tf.float32)
    ph_y = tf.placeholder(tf.int32)
    ph_keep = tf.placeholder(tf.float32)

    # ---------------------------------------------------------------------------------------------------------#
    # truncated_normal : 절단 정규 분포, 양쪽 끝을 짜른것 → 테스트시 결과가 잘나온다, stddev=0.1 편차가 작으면 좋다
    # → xavier = glorot을 최근에 사용함으로 의미가 없다.→
    # weight가 많으면 조절하기 쉽다(음악 equalizer 처럼) → 편차가 조절하는것이 좋다.
    # 어떤 특정 weight를 높이는 것보다는  모드 weight의 편차를 올려  특정 weight가 높아지는것보다 좋다

    # tensorflow reshape vs numpy reshape
    # tensorflow의 reshape는 데이터가 없고, numpy는 데이터가 있다의 차이
    # tensorflow reshape는 나중에 session에 들어가서 사용, numpy로 될부분은 당연히 numpy의  reshape을 사용
    x_image = tf.reshape(ph_x, [-1, 28, 28, 1]) # 784 28 * 28  만약 컬러면 1이 아니고 3

    # [3,3 → Filter의미, ,1 → 흑백 channel 한개,  32] → 32개의 Filter
    w_conv_1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1)) # channel의 1 → 32로 증가한다(두꺼워진다)
    b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

    # 모델 구성
    # l은 layer라는 의미, 1은  첫번째 layer라는 뜻
    # strides의  양쪽1은 사용 하지 않음, 가운데 1(수평), 1(수직)만 사용함.
    # x_image : 데이터
    # w_conv_1 : filter의미
    # 28 * 28의 크기가 줄어들기를 바라지 않아서 'SAME' option  만약 # VALID는 zero padding을 하지 않겠다.
    # ksize[1, 2, 2, 1] : 양쪽 1은 사용하지 않음  가운데 2,2만 사용,  4차원을 맞추기기위해 양쪽 1을 사용함
    # (?, 28, 28, 32)
    l_conv_1 = tf.nn.conv2d(x_image, w_conv_1, strides=[1, 1, 1, 1], padding='SAME')
    l_relu_1 = tf.nn.relu(l_conv_1 + b_conv_1)
    # (?, 14, 14, 32) ksize=커널 사이즈, 아래는 SAME을 쓰나 VALID를 쓰나 차이가 없다
    l_pool_1 = tf.nn.max_pool(l_relu_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 4차원 Data가  → 2차원 Data로 변경되어야 한다, 3차원을 → 1차원
    w_conv_2 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev=0.1))
    b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))

    # (?, 14, 14, 64)
    l_conv_2 = tf.nn.conv2d(l_pool_1, w_conv_2, strides=[1, 1, 1, 1], padding='SAME')
    l_relu_2 = tf.nn.relu(l_conv_2 + b_conv_2)
    # (?, 7, 7, 64)
    l_pool_2 = tf.nn.max_pool(l_relu_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # (?, 7 * 7 * 64)
    # [-1, 7 * 7 * 64]   -1은 어떤 숫자인지 몰라서,  100은 학습,  나중에 batch_size가 10000개가 될지 모른다.
    l_flat   = tf.reshape(l_pool_2, [-1, 7 * 7 * 64])

    # 28 → 절반 줄여서 14 → 또 절반 줄여서 7
    # [7 * 7 * 64 →  3차원을 1차원으로
    w_full_3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 256], stddev=0.1))
    b_full_3 = tf.Variable(tf.constant(0.1, shape=[256]))

    # (?, 256)
    l_full_3 = tf.matmul(l_flat, w_full_3) + b_full_3
    l_relu_3 = tf.nn.relu(l_full_3)
    l_drop_3 = tf.nn.dropout(l_relu_3, ph_keep)

    w_full_4 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
    b_full_4 = tf.Variable(tf.constant(0.1, shape=[10]))

    # (?, 10)
    z = tf.matmul(l_drop_3, w_full_4) + b_full_4
    hx = tf.nn.softmax(z)

    print('l_conv_1', l_conv_1.shape)
    print('l_pool_1', l_pool_1.shape)
    print('l_conv_2', l_conv_2.shape)
    print('l_pool_2', l_pool_2.shape)
    print('l_flat',   l_flat.shape)
    print('l_full_3', l_full_3.shape)
    print('z',        z.shape)

    return

    # ---------------------------------------------------------------------------------------------------------#

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=ph_y)
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  → 빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 1
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, yy = mnist.train.next_batch(batch_size)

            sess.run(train, feed_dict={ph_x: xx, ph_y: yy, ph_keep: 0.5})
            c += sess.run(loss, {ph_x: xx, ph_y:yy, ph_keep: 0.5})

        print(i, c/ n_iters)

    preds = sess.run(hx, {ph_x: mnist.test.images, ph_keep: 1.0})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    # test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == mnist.test.labels))

    print(preds_arg)
    # print(test_arg)

    sess.close()


# 문제
# One-hot label을 사용하고, 입력에 대해 reshape 하지 마세요.

In [3]:
def mnist_cnn_2():
    mnist = input_data.read_data_sets('mnist', one_hot=True)

    ph_x = tf.placeholder(tf.float32)
    ph_y = tf.placeholder(tf.int32)
    ph_keep = tf.placeholder(tf.float32)

    # ---------------------------------------------------------------------------------------------------------#
    # truncated_normal : 절단 정규 분포, 양쪽 끝을 짜른것 → 테스트시 결과가 잘나온다, stddev=0.1 편차가 작으면 좋다
    # → xavier = glorot을 최근에 사용함으로 의미가 없다.→
    # weight가 많으면 조절하기 쉽다(음악 equalizer 처럼) → 편차가 조절하는것이 좋다.
    # 어떤 특정 weight를 높이는 것보다는  모드 weight의 편차를 올려  특정 weight가 높아지는것보다 좋다

    # [3,3 → Filter의미, ,1 → 흑백 channel 한개,  32] → 32개의 Filter
    w_conv_1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1)) # channel의 1 → 32로 증가한다(두꺼워진다)
    b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

    # 모델 구성
    # l은 layer라는 의미, 1은  첫번째 layer라는 뜻
    # strides의  양쪽1은 사용 하지 않음, 가운데 1(수평), 1(수직)만 사용함.
    # x_image : 데이터
    # w_conv_1 : filter의미
    # 28 * 28의 크기가 줄어들기를 바라지 않아서 'SAME' option  만약 # VALID는 zero padding을 하지 않겠다.
    # ksize[1, 2, 2, 1] : 양쪽 1은 사용하지 않음  가운데 2,2만 사용,  4차원을 맞추기기위해 양쪽 1을 사용함
    # (?, 28, 28, 32)
    l_conv_1 = tf.nn.conv2d(ph_x, w_conv_1, strides=[1, 1, 1, 1], padding='SAME')
    l_relu_1 = tf.nn.relu(l_conv_1 + b_conv_1)
    # (?, 14, 14, 32) ksize=커널 사이즈, 아래는 SAME을 쓰나 VALID를 쓰나 차이가 없다
    l_pool_1 = tf.nn.max_pool(l_relu_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # 4차원 Data가  → 2차원 Data로 변경되어야 한다, 3차원을 → 1차원
    w_conv_2 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev=0.1))
    b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))

    # (?, 14, 14, 64)
    l_conv_2 = tf.nn.conv2d(l_pool_1, w_conv_2, strides=[1, 1, 1, 1], padding='SAME')
    l_relu_2 = tf.nn.relu(l_conv_2 + b_conv_2)
    # (?, 7, 7, 64)
    l_pool_2 = tf.nn.max_pool(l_relu_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # (?, 7 * 7 * 64)
    # [-1, 7 * 7 * 64]   -1은 어떤 숫자인지 몰라서,  100은 학습,  나중에 batch_size가 10000개가 될지 모른다.
    l_flat   = tf.reshape(l_pool_2, [-1, 7 * 7 * 64])

    # 28 → 절반 줄여서 14 → 또 절반 줄여서 7
    # [7 * 7 * 64 →  3차원을 1차원으로
    w_full_3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 256], stddev=0.1))
    b_full_3 = tf.Variable(tf.constant(0.1, shape=[256]))

    # (?, 256)
    l_full_3 = tf.matmul(l_flat, w_full_3) + b_full_3
    l_relu_3 = tf.nn.relu(l_full_3)
    l_drop_3 = tf.nn.dropout(l_relu_3, ph_keep)

    w_full_4 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
    b_full_4 = tf.Variable(tf.constant(0.1, shape=[10]))

    # (?, 10)
    z = tf.matmul(l_drop_3, w_full_4) + b_full_4
    hx = tf.nn.softmax(z)

    print('l_conv_1', l_conv_1.shape)
    print('l_pool_1', l_pool_1.shape)
    print('l_conv_2', l_conv_2.shape)
    print('l_pool_2', l_pool_2.shape)
    print('l_flat',   l_flat.shape)
    print('l_full_3', l_full_3.shape)
    print('z',        z.shape)

    # return

    # ---------------------------------------------------------------------------------------------------------#

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=z, labels=ph_y)
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 1
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, yy = mnist.train.next_batch(batch_size)

            xx = xx.reshape([-1, 28, 28, 1])

            sess.run(train, feed_dict={ph_x: xx, ph_y: yy, ph_keep: 0.5})
            c += sess.run(loss, {ph_x: xx, ph_y:yy, ph_keep: 0.5})

        print(i, c/ n_iters)

    xx = mnist.test.images.reshape([-1, 28, 28, 1])
    preds = sess.run(hx, {ph_x: xx, ph_keep: 1.0})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == test_arg))

    print(preds_arg)
    # print(test_arg)

    sess.close()

In [ ]:
mnist_cnn_2()